In [5]:
import numpy as np
import pandas as pd
import random
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
# Seed 고정 함수
'''
[Why?]

- 딥러닝은 초기 가중치, 배치 순서 등에 따라 매번 결과가 달라짐
- Seed를 고정하면 '재현 가능한 실험' 가능
- 이후 여러 Seed를 사용하면 서로 다른 관점의 모델 앙상블 가능
'''

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [6]:
# 데이터 로드
BASE_DIR = Path.cwd().parent.parent
DATA_DIR = BASE_DIR / 'data:raw'

train_df = pd.read_csv(DATA_DIR / 'train.csv')
test_df = pd.read_csv(DATA_DIR / 'test_x.csv')

In [7]:
# Target 값 정리 (1, 2 -> 0, 1)
TARGET = 'voted'
train_df[TARGET] = train_df[TARGET] - 1

In [8]:
# Feature / Column 분리
'''
[칼럼 분리 이유]

- target은 학습 입력에서 제거
- 수치형 / 범주형 분리
 -> 딥러닝은 수치형 처리에 강점
 -> 범주형은 이번 baseline에서는 제외
'''

DROP_COLS = [TARGET]
FEATURES = [col for col in train_df.columns if col not in DROP_COLS]

NUM_COLS = train_df[FEATURES].select_dtypes(include=['int64', 'float64']).columns.tolist()
CAT_COLS = train_df[FEATURES].select_dtypes(include=['object', 'category']).columns.tolist()

In [9]:
# 전처리 (결측치 + 스케일링)
'''
[전처리 전략]

1. 결측치 처리
    - 수치형: median
     -> 이상치에 강건

2. 표준화(StandardScaler)
    - 딥러닝은 feature scale에 민감
    - 평균 0, 분산 1로 맞춤

3. 범주형 칼럼 제거
    - 현재는 NN baseline
    - 이후 embedding으로 확장 가능
'''

num_imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

# 수치형 결측치 처리
train_df[NUM_COLS] = num_imputer.fit_transform(train_df[NUM_COLS])
test_df[NUM_COLS] = num_imputer.transform(test_df[NUM_COLS])

# 스케일링
train_df[NUM_COLS] = scaler.fit_transform(train_df[NUM_COLS])
test_df[NUM_COLS] = scaler.transform(test_df[NUM_COLS])

# 범주형 제거
train_df = train_df[NUM_COLS + [TARGET]]
test_df = test_df[NUM_COLS]

In [10]:
# Train / Validation 분리
'''
[Why?]

- AUC는 threshold-independent metric
- 학습 중 '일반화 성능'을 확인해야 함
- strarify=y:
 -> 클래스 비율 유지 (불균형 방지)
'''

X = train_df.drop(columns=[TARGET]).values
y = train_df[TARGET].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [11]:
# Pytorch Dataset 정의
'''
[Dataset 역할]

- numpy -> torch tensor 변환
- DataLoader와 결합되어 mini-batch 학습 가능
'''

class TabularDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is None:
            return self.X[idx]
        return self.X[idx], self.y[idx]

In [12]:
# 딥러닝 모델 정의 (AUC 최적화 구조)
'''
[모델 설계 의도]

- 깊지 않지만 충분한 표현력
- BatchNorm + Dropout
 -> 과적합 방지
- 마지막 출력은 logit (sigmoid X)
 -> BCEWithLogitsLoss와 결합
'''

class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)

In [17]:
# 단일 Seed 학습 함수
'''
[핵심 함수]
- 하나의 seed로 모델 학습
- Validation AUC 기준으로 best model 저장
- class imbalance -> pos_weight 적용
'''

def train_one_seed(seed):
    set_seed(seed)

    model = MLP(X_train.shape[1])

    optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=1e-3
    )

# 클래스 불균형 보정
    pos = y_train.sum()
    neg = len(y_train) - pos
    pos_weight = torch.tensor([neg / pos])

    criterion = nn.BCEWithLogitsLoss(
        pos_weight=pos_weight
    )

    train_loader = DataLoader(
    TabularDataset(X_train, y_train),
    batch_size=256,
    shuffle=True
    )

    val_loader = DataLoader(
    TabularDataset(X_val, y_val),
    batch_size=512,
    shuffle=False
    )

    best_auc = 0.0
    best_state = None

    for epoch in range(30):
    #--1.Train--
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

    # --2.Validataion--
        model.eval()
        preds = []

        with torch.no_grad():
            for xb, yb in val_loader:
                preds.extend(
                    torch.sigmoid(model(xb)).cpu().numpy()
                )

        auc = roc_auc_score(y_val, preds)

    # --3. Best model 저장--
        if auc > best_auc:
            best_auc = auc
            best_state = model.state_dict()

        print(f'[Seed {seed}] Epoch {epoch+1:02d} | Val AUC: {auc:.5f}')

    model.load_state_dict(best_state)
    return model

In [18]:
# Seed 앙상블
'''
[Why?]
- 같은 구조라도 seed가 다르면 서로 다른 decision boundary 학습
- 확률 평균 -> 분산 감소
- AUC 상승에 매우 효과적
'''

SEEDS = [0, 1, 2, 3, 4]
test_loader = DataLoader(
    TabularDataset(test_df.values),
    batch_size=512
)

all_test_probs = []

for seed in SEEDS:
    model = train_one_seed(seed)
    model.eval()

    probs =[]
    with torch.no_grad():
        for xb in test_loader:
            probs.extend(torch.sigmoid(model(xb)).cpu().numpy())

    all_test_probs.append(np.array(probs))

[Seed 0] Epoch 01 | Val AUC: 0.72318
[Seed 0] Epoch 02 | Val AUC: 0.72940
[Seed 0] Epoch 03 | Val AUC: 0.73244
[Seed 0] Epoch 04 | Val AUC: 0.73418
[Seed 0] Epoch 05 | Val AUC: 0.73691
[Seed 0] Epoch 06 | Val AUC: 0.73710
[Seed 0] Epoch 07 | Val AUC: 0.73632
[Seed 0] Epoch 08 | Val AUC: 0.73693
[Seed 0] Epoch 09 | Val AUC: 0.73866
[Seed 0] Epoch 10 | Val AUC: 0.73809
[Seed 0] Epoch 11 | Val AUC: 0.74041
[Seed 0] Epoch 12 | Val AUC: 0.73856
[Seed 0] Epoch 13 | Val AUC: 0.73798
[Seed 0] Epoch 14 | Val AUC: 0.73774
[Seed 0] Epoch 15 | Val AUC: 0.73777
[Seed 0] Epoch 16 | Val AUC: 0.74106
[Seed 0] Epoch 17 | Val AUC: 0.73793
[Seed 0] Epoch 18 | Val AUC: 0.73966
[Seed 0] Epoch 19 | Val AUC: 0.73793
[Seed 0] Epoch 20 | Val AUC: 0.73954
[Seed 0] Epoch 21 | Val AUC: 0.74040
[Seed 0] Epoch 22 | Val AUC: 0.74007
[Seed 0] Epoch 23 | Val AUC: 0.73895
[Seed 0] Epoch 24 | Val AUC: 0.73889
[Seed 0] Epoch 25 | Val AUC: 0.73686
[Seed 0] Epoch 26 | Val AUC: 0.73731
[Seed 0] Epoch 27 | Val AUC: 0.73792
[

In [19]:
final_prob = np.mean(all_test_probs, axis=0)

submission = pd.read_csv(DATA_DIR / 'sample_submission.csv')

submission['voted'] = final_prob

submission.to_csv('submit_260128_hana03.csv', index=False)
print('❗️ submission.csv 저장 완료')

❗️ submission.csv 저장 완료
